In [30]:
from simulate_wizard import Game
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

distribtions = []

for n_players in range(3, 8):
    for n_cards in range(1,20):
        if n_cards * n_players <= 59:
            results = []
            for _ in range(10_000):
                a_game = Game(n_players=n_players,n_cards=n_cards)
                res = a_game.compute_results()
                results.extend(res)

            save_name = f'data/records_{n_players}_{n_cards}.csv'
            df = pd.DataFrame.from_records(results)
            df.to_csv(save_name)
            distribtions.append((n_cards,n_players, df['n_won'].value_counts(normalize=True).values*100))

In [32]:
df = pd.DataFrame.from_records(distribtions)

In [37]:
df.columns = ['n_cards','n_players', 'dist']
df[df['n_players'] == 4]

,n_cards,n_players,dist
19,1,4,"[75.0, 25.0]"
20,2,4,"[58.8, 32.4, 8.799999999999999]"
21,3,4,"[46.805, 35.05, 14.485000000000001, 3.66]"
22,4,4,"[37.3125, 35.655, 18.29, 7.205, 1.5375]"
23,5,4,"[34.2375, 30.2475, 21.0225, 10.035, 3.675, 0.7..."
24,6,4,"[32.085, 24.635, 22.735, 12.2775, 5.975, 2.042..."
25,7,4,"[30.2125, 23.48, 19.79, 14.057500000000001, 7...."
26,8,4,"[27.66, 23.727500000000003, 16.342499999999998..."
27,9,4,"[24.4725, 23.985, 16.86, 13.444999999999999, 1..."
28,10,4,"[23.572499999999998, 22.445, 17.285, 11.584999..."


In [40]:
import glob

dfs_with_5_players = glob.glob("data/records_5*")



['data/records_5_10.csv',
 'data/records_5_2.csv',
 'data/records_5_6.csv',
 'data/records_5_4.csv',
 'data/records_5_7.csv',
 'data/records_5_9.csv',
 'data/records_5_11.csv',
 'data/records_5_3.csv',
 'data/records_5_1.csv',
 'data/records_5_8.csv',
 'data/records_5_5.csv']

# Answer questions

Given amoung hands with a wizard what is the optimal bet?

In [92]:
results = []
for _ in range(10_000):
                a_game = Game(n_players=5,n_cards=4)
                res = a_game.compute_results()
                results.extend(res)

df = pd.DataFrame.from_records(results)
df

,n_players,n_cards,n_won,trump,hand
0,5,4,1,spades,"[10-spades, 12-diamonds, 9-diamonds, 14-hearts]"
1,5,4,0,spades,"[7-spades, 13-diamonds, 7-hearts, 7-diamonds]"
2,5,4,2,spades,"[8-spades, wizard, 10-clubs, 6-spades]"
3,5,4,0,spades,"[14-diamonds, wizard, 5-hearts, 11-clubs]"
4,5,4,1,spades,"[12-spades, 5-clubs, 2-spades, 4-clubs]"
...,...,...,...,...,...
49995,5,4,2,diamonds,"[9-clubs, 5-spades, 12-hearts, 13-hearts]"
49996,5,4,0,diamonds,"[jester, 2-hearts, 6-clubs, 2-clubs]"
49997,5,4,1,diamonds,"[3-hearts, 2-diamonds, jester, 5-clubs]"
49998,5,4,0,diamonds,"[9-spades, 4-clubs, 11-spades, wizard]"


In [93]:
def count_trump(hand, trump):
    # might want to opimize
    n_trump = 0
    for card in hand:
        if trump == card.suit:
            n_trump +=1
    return n_trump

a_hand = df['hand'].values[0]
count_trump(a_hand, 'spades')

1

In [94]:
%%time
df['n_trump'] = df.apply(lambda row: count_trump(hand=row['hand'], trump=row['trump']), axis=1)

CPU times: user 345 ms, sys: 0 ns, total: 345 ms
Wall time: 344 ms


In [95]:
df['n_trump'].value_counts()

1    21083
0    20111
2     7712
3     1054
4       40
Name: n_trump, dtype: int64

In [101]:
outcomes_when_one_trump = df[df['n_trump']==2]
outcomes_when_one_trump['n_won'].value_counts(normalize=True).sort_index()

0    0.150156
1    0.419865
2    0.322355
3    0.089860
4    0.017765
Name: n_won, dtype: float64